In [22]:
import os
import getpass
import pandas as pd
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
import openpyxl
from preprocess import clean_unnamed_header, fill_undefined_sequentially, forward_fill_column_nans
from utils import read_file, get_feature_name_content, format_nested_dict_for_llm
from metadata import get_number_of_row_header, convert_df_headers_to_nested_dict, convert_df_rows_to_nested_dict
from llm import get_schema, get_feature_names
import dotenv
dotenv.load_dotenv()

# Initialize the Gemini 2.5 Flash model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-preview-04-17",
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

file_path = '../data/example2.xlsx'

In [34]:
feature_names = get_feature_names(file_path, llm)
_, feature_name_result = get_feature_name_content(file_path, feature_names)
number_of_row_header = get_number_of_row_header(file_path)
df = pd.read_excel(file_path, header=list(range(0,number_of_row_header)))
df = clean_unnamed_header(df, number_of_row_header)
df = fill_undefined_sequentially(df,feature_name_result["feature_rows"])
df = forward_fill_column_nans(df,feature_name_result["feature_rows"])


### Thinking
The provided content has four header rows. The first row lists potential feature names (`TT`, `Chỉ tiêu`, `Số liệu`, `Phân loại`, `Loại hình`, `Năm 2024`, etc.). The subsequent header rows provide specific values and aggregations (`6 Tháng đầu năm`, `Quý I`, `Quý II`, `Tháng 01`, `Tháng 02`, `Cộng`, `Tổng cộng`, etc.) under the columns starting from `Năm 2024`. This indicates that the columns from `Năm 2024` onwards represent feature columns with a hierarchical structure based on time periods and totals.

The initial columns (`Chỉ tiêu`, `Số liệu`, `Phân loại`, `Loại hình`) show a clear hierarchical structure in the data rows, where values are nested within the preceding columns (e.g., `Kế hoạch (Tập đoàn)` is under `DOANH THU`, `Sản xuất` is under `Kế hoạch (Tập đoàn)`). This structure defines the rows of the table.

The combination of multiple header rows defining feature columns (time periods, totals) and hierarchical structure in the initial columns defining the rows (

d:\project\vdt\xlsx_chatbot\script\preprocess.py:59: PerformanceWarning: indexing past lexsort depth may impact performance.
  df[column_name] = df[column_name].ffill()
d:\project\vdt\xlsx_chatbot\script\preprocess.py:59: PerformanceWarning: indexing past lexsort depth may impact performance.
  df[column_name] = df[column_name].ffill()
d:\project\vdt\xlsx_chatbot\script\preprocess.py:59: PerformanceWarning: indexing past lexsort depth may impact performance.
  df[column_name] = df[column_name].ffill()
d:\project\vdt\xlsx_chatbot\script\preprocess.py:59: PerformanceWarning: indexing past lexsort depth may impact performance.
  df[column_name] = df[column_name].ffill()


In [35]:
df

TT                Chỉ tiêu               Số liệu  \
     Header                  Header                Header   
     Header                  Header                Header   
     Header                  Header                Header   
0         1               DOANH THU             Undefined   
1       1.1               DOANH THU             Undefined   
2     1.1.1               DOANH THU             Undefined   
3     1.1.2               DOANH THU             Undefined   
4     1.1.3               DOANH THU             Undefined   
..      ...                     ...                   ...   
176  14.1.3  Nộp ngân sách Nhà nước      Tổng số phải nộp   
177    14.2  Nộp ngân sách Nhà nước  Số nộp tại Thanh Hóa   
178      15            Tổng tài sản             Undefined   
179      16          Vốn chủ sở hữu             Undefined   
180      17                 Vốn vay             Undefined   

               Phân loại   Loại hình        Năm 2024                         \
                  Header      Header 6 Tháng đầu năm                          
                  Header      Header           Quý I                          
                  Header      Header        Tháng 01 Tháng 02 Tháng 03 Cộng   
0              Undefined   Undefined             NaN      NaN      NaN  NaN   
1    Kế hoạch (Tập đoàn)   Undefined             NaN      NaN      NaN  NaN   
2    Kế hoạch (Tập đoàn)    Sản xuất             NaN      NaN      NaN  NaN   
3    Kế hoạch (Tập đoàn)     Dịch vụ             NaN      NaN      NaN  NaN   
4    Kế hoạch (Tập đoàn)  Thương mại             NaN      NaN      NaN  NaN   
..                   ...         ...             ...      ...      ...  ...   
176         % Hoàn thành   Undefined             NaN      NaN      NaN  NaN   
177            Undefined   Undefined             NaN      NaN      NaN  NaN   
178            Undefined   Undefined             NaN      NaN      NaN  NaN   
179            Undefined   Undefined             NaN      NaN      NaN  NaN   
180            Undefined   Undefined             NaN      NaN      NaN  NaN   

              ...                                                             \
              ...                                                              
      Quý II  ...  Quý III                          Quý IV                     
    Tháng 04  ... Tháng 07 Tháng 08 Tháng 09 Cộng Tháng 10 Tháng 11 Tháng 12   
0        NaN  ...      NaN      NaN      NaN  NaN      NaN      NaN      NaN   
1        NaN  ...      NaN      NaN      NaN  NaN      NaN      NaN      NaN   
2        NaN  ...      NaN      NaN      NaN  NaN      NaN      NaN      NaN   
3        NaN  ...      NaN      NaN      NaN  NaN      NaN      NaN      NaN   
4        NaN  ...      NaN      NaN      NaN  NaN      NaN      NaN      NaN   
..       ...  ...      ...      ...      ...  ...      ...      ...      ...   
176      NaN  ...      NaN      NaN      NaN  NaN      NaN      NaN      NaN   
177      NaN  ...      NaN      NaN      NaN  NaN      NaN      NaN      NaN   
178      NaN  ...      NaN      NaN      NaN  NaN      NaN      NaN      NaN   
179      NaN  ...      NaN      NaN      NaN  NaN      NaN      NaN      NaN   
180      NaN  ...      NaN      NaN      NaN  NaN      NaN      NaN      NaN   

                           
                Tổng cộng  
           Cộng    Header  
    Cộng Header    Header  
0    NaN    NaN       NaN  
1    NaN    NaN       NaN  
2    NaN    NaN       NaN  
3    NaN    NaN       NaN  
4    NaN    NaN       NaN  
..   ...    ...       ...  
176  NaN    NaN       NaN  
177  NaN    NaN       NaN  
178  NaN    NaN       NaN  
179  NaN    NaN       NaN  
180  NaN    NaN       NaN  

[181 rows x 24 columns]

In [36]:
feature_name_result["feature_rows"]

['Chỉ tiêu', 'Số liệu', 'Phân loại', 'Loại hình']

In [37]:
feature_name_result["feature_cols"]

['Năm 2024']

In [5]:
row_dict = convert_df_rows_to_nested_dict(df, feature_name_result["feature_rows"])
row_dict

{'DOANH THU': {'Undefined': {'Kế hoạch (Tập đoàn)': ['Sản xuất',
    'Dịch vụ',
    'Thương mại',
    'Khác'],
   'Thực hiện': ['Sản xuất', 'Dịch vụ', 'Thương mại', 'Khác'],
   '% Hoàn thành': ['Sản xuất', 'Dịch vụ', 'Thương mại', 'Khác']}},
 'Lợi nhuận gộp': {'Tổng lợi nhuận gộp': {'Kế hoạch (Tập đoàn)': ['Sản xuất',
    'Dịch vụ',
    'Thương mại',
    'Khác'],
   'Thực hiện': ['Sản xuất', 'Dịch vụ', 'Thương mại', 'Khác'],
   '% Hoàn thành': ['Sản xuất', 'Dịch vụ', 'Thương mại', 'Khác']},
  'Tỷ suất trên Doanh thu': {'Kế hoạch (Tập đoàn)': ['Sản xuất',
    'Dịch vụ',
    'Thương mại',
    'Khác'],
   'Thực hiện': ['Sản xuất', 'Dịch vụ', 'Thương mại', 'Khác'],
   '% Hoàn thành': ['Sản xuất', 'Dịch vụ', 'Thương mại', 'Khác']}},
 'Chi phí bán hàng và Chi phí quản lý': {'Tổng chi phí bán hàng và Chi phí quản lý Công ty': {'Kế hoạch (Tập đoàn)': ['Sản xuất',
    'Dịch vụ',
    'Thương mại',
    'Khác'],
   'Thực hiện': ['Sản xuất', 'Dịch vụ', 'Thương mại', 'Khác'],
   '% Hoàn thành': ['Sả

In [38]:
col_dict = convert_df_headers_to_nested_dict(df, feature_name_result["feature_cols"])
col_dict

{'Năm 2024': {'6 Tháng đầu năm': {'Quý I': ['Cộng',
    'Tháng 01',
    'Tháng 02',
    'Tháng 03'],
   'Quý II': ['Cộng', 'Tháng 04', 'Tháng 05', 'Tháng 06'],
   'Cộng': {},
   'Quý III': ['Cộng', 'Tháng 07', 'Tháng 08', 'Tháng 09'],
   'Quý IV': ['Cộng', 'Tháng 10', 'Tháng 11', 'Tháng 12']},
  'Tổng cộng': {}}}

In [39]:
# col_dict_llm_content =  format_nested_dict_for_llm(col_dict)
# row_dict_llm_content =  format_nested_dict_for_llm(row_dict)

In [40]:
feature_cols = feature_name_result["feature_cols"]
feature_cols

['Năm 2024']

In [41]:
print(row_dict)

{'DOANH THU': {'Undefined': {'Kế hoạch (Tập đoàn)': ['Sản xuất', 'Dịch vụ', 'Thương mại', 'Khác'], 'Thực hiện': ['Sản xuất', 'Dịch vụ', 'Thương mại', 'Khác'], '% Hoàn thành': ['Sản xuất', 'Dịch vụ', 'Thương mại', 'Khác']}}, 'Lợi nhuận gộp': {'Tổng lợi nhuận gộp': {'Kế hoạch (Tập đoàn)': ['Sản xuất', 'Dịch vụ', 'Thương mại', 'Khác'], 'Thực hiện': ['Sản xuất', 'Dịch vụ', 'Thương mại', 'Khác'], '% Hoàn thành': ['Sản xuất', 'Dịch vụ', 'Thương mại', 'Khác']}, 'Tỷ suất trên Doanh thu': {'Kế hoạch (Tập đoàn)': ['Sản xuất', 'Dịch vụ', 'Thương mại', 'Khác'], 'Thực hiện': ['Sản xuất', 'Dịch vụ', 'Thương mại', 'Khác'], '% Hoàn thành': ['Sản xuất', 'Dịch vụ', 'Thương mại', 'Khác']}}, 'Chi phí bán hàng và Chi phí quản lý': {'Tổng chi phí bán hàng và Chi phí quản lý Công ty': {'Kế hoạch (Tập đoàn)': ['Sản xuất', 'Dịch vụ', 'Thương mại', 'Khác'], 'Thực hiện': ['Sản xuất', 'Dịch vụ', 'Thương mại', 'Khác'], '% Hoàn thành': ['Sản xuất', 'Dịch vụ', 'Thương mại', 'Khác']}, 'Tỷ suất trên Doanh thu': {'Kế h

In [42]:
feature_rows = feature_name_result["feature_rows"]
feature_rows

['Chỉ tiêu', 'Số liệu', 'Phân loại', 'Loại hình']

In [43]:
print(col_dict)

{'Năm 2024': {'6 Tháng đầu năm': {'Quý I': ['Cộng', 'Tháng 01', 'Tháng 02', 'Tháng 03'], 'Quý II': ['Cộng', 'Tháng 04', 'Tháng 05', 'Tháng 06'], 'Cộng': {}, 'Quý III': ['Cộng', 'Tháng 07', 'Tháng 08', 'Tháng 09'], 'Quý IV': ['Cộng', 'Tháng 10', 'Tháng 11', 'Tháng 12']}, 'Tổng cộng': {}}}


In [46]:
query = "Cho tôi biết số tiền Nộp ngân sách Nhà nước tổng cộng là bao nhiêu ở quý 1"

In [47]:
from llm import process_natural_language_query
result = process_natural_language_query(query, feature_rows, feature_cols, row_dict, col_dict, llm)

LLM Response:
### Thinking:
**1. Query Analysis:**
The user's request is: "Cho tôi biết số tiền Nộp ngân sách Nhà nước tổng cộng là bao nhiêu ở quý 1".
This translates to: Show the amount for "Nộp ngân sách Nhà nước" (State budget submission), specifically the "tổng cộng" (total) amount, for "quý 1" (Quarter 1).
- Row-related keywords: "Nộp ngân sách Nhà nước", "tổng cộng", "số tiền" (implies 'Thực hiện').
- Column-related keywords: "quý 1".

**2. Row Tracing (Adhering to Instruction 2):**
The `Feature Rows` are `['Chỉ tiêu', 'Số liệu', 'Phân loại', 'Loại hình']`.
- For 'Chỉ tiêu': The query specifies "Nộp ngân sách Nhà nước". This maps directly to `'Chỉ tiêu': ['Nộp ngân sách Nhà nước']`.
- For 'Số liệu': The query specifies "tổng cộng", which maps to "Tổng số phải nộp" under "Nộp ngân sách Nhà nước" in the `Row Hierarchy`. So, `'Số liệu': ['Tổng số phải nộp']`.
- For 'Phân loại': The query asks for "số tiền", which implies the actual value, mapping to "Thực hiện" under "Tổng số phải 